In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR

In [2]:
cleared_data_all = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset.xlsx')
cleared_data_all = cleared_data_all.iloc[:, 1:]
cleared_data_all

,Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2015 US$),Adjusted net national income per capita (current US$),"Adjusted net savings, excluding particulate emission damage (current US$)","Adjusted net savings, including particulate emission damage (current US$)",Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: consumption of fixed capital (current US$),...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.541305,0.018580,0.001411,0.493338,0.255733,0.024477,0.042788,0.041910,7.641349e-07,0.000160,...,0.109494,0.006858,0.098921,0.047494,0.284959,0.366300,0.211988,0.381785,0.440389,0.443970
1,0.542030,0.011485,0.001591,0.497128,0.257579,0.031381,0.043985,0.044182,1.141130e-04,0.000511,...,0.061274,0.036062,0.090457,0.054871,0.267499,0.335687,0.295060,0.331167,0.353970,0.359865
2,0.532645,0.013996,0.002015,0.486800,0.309672,0.028568,0.042789,0.046088,4.111755e-05,0.000642,...,0.056934,0.048997,0.112458,0.038223,0.235780,0.293609,0.236735,0.276357,0.283570,0.325602
3,0.529905,0.021782,0.001641,0.513873,0.335287,0.027832,0.042536,0.043784,1.076366e-04,0.000626,...,0.073445,0.042937,0.100149,0.045912,0.309495,0.269240,0.292649,0.269380,0.402339,0.451430
4,0.523391,0.015874,0.001086,0.505739,0.319546,0.025767,0.042422,0.048538,1.100043e-04,0.000375,...,0.052960,0.027781,0.075377,0.038749,0.300016,0.216947,0.173194,0.303925,0.367756,0.463100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.430820,0.016480,0.019676,0.381147,0.222597,0.355406,0.037475,0.030756,3.170770e-02,0.018601,...,0.598647,0.633237,0.791004,0.608203,0.052758,0.094311,0.103817,0.493570,0.648812,0.429089
2028,0.431366,0.016379,0.015311,0.380339,0.222985,0.235515,0.037528,0.030807,2.144808e-02,0.015013,...,0.581060,0.480167,0.845445,0.906866,0.054837,0.096322,0.085744,0.495072,0.788197,0.621088
2029,0.430798,0.016389,0.017261,0.382386,0.324990,0.292323,0.037512,0.030619,2.867491e-02,0.019105,...,0.652924,0.659372,0.775232,0.769994,0.060454,0.098751,0.085432,0.472306,0.549884,0.540978
2030,0.430458,0.016307,0.023369,0.381875,0.328690,0.283576,0.037511,0.030659,3.792288e-02,0.022233,...,0.623938,0.697836,0.750916,0.541441,0.071787,0.080619,0.092265,0.654687,0.516524,0.400289


In [3]:
cleared_data_only_percent = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset_only_%.xlsx')
cleared_data_only_percent = cleared_data_only_percent.iloc[:, 1:]
cleared_data_only_percent

,Adjusted net national income (annual % growth),Adjusted net national income per capita (annual % growth),"Agriculture, forestry, and fishing, value added (annual % growth)",Chemicals (% of value added in manufacturing),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Debt service (PPG and IMF only, % of exports of goods, services and primary income)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Food, beverages and tobacco (% of value added in manufacturing)",...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.553663,0.493048,0.550624,0.659424,0.482240,0.204614,0.370799,0.527068,1.000000,0.332565,...,0.092583,0.044941,0.116194,0.058954,0.271638,0.256557,0.236409,0.340039,0.213734,0.288946
1,0.540093,0.520842,0.572925,0.515892,0.579728,0.170912,0.409697,0.533898,0.949101,0.361794,...,0.080344,0.045471,0.119967,0.067618,0.240226,0.257459,0.168916,0.451473,0.356074,0.258655
2,0.535596,0.540533,0.599002,0.593352,0.517354,0.204310,0.433730,0.517269,0.943975,0.278636,...,0.073603,0.050711,0.072459,0.041396,0.288064,0.289370,0.211341,0.448748,0.419979,0.258733
3,0.526273,0.508690,0.638806,0.523286,0.468628,0.196385,0.342814,0.533692,0.838995,0.384575,...,0.077985,0.041490,0.144534,0.034277,0.245759,0.284184,0.213594,0.461372,0.347358,0.254320
4,0.549766,0.500511,0.690262,0.515493,0.394821,0.132378,0.330198,0.532333,0.961634,0.409441,...,0.068681,0.047989,0.115199,0.022896,0.237920,0.279386,0.248200,0.383103,0.306070,0.305546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.422859,0.408177,0.546620,0.209895,0.347769,0.502068,0.172372,0.378021,0.333544,0.478746,...,0.842023,0.675799,0.880286,0.653719,0.101137,0.068454,0.085621,0.401947,0.498277,0.469291
2028,0.422482,0.407757,0.569233,0.023101,0.358500,0.622407,0.193755,0.358328,0.317363,0.507215,...,0.957228,0.608725,0.816423,0.536649,0.112019,0.073421,0.059600,0.339091,0.414780,0.576167
2029,0.422558,0.409683,0.625547,0.099384,0.356030,0.667137,0.170789,0.352184,0.282537,0.640014,...,0.841818,0.642024,0.733566,0.716802,0.087615,0.078230,0.059868,0.450768,0.445414,0.564287
2030,0.423154,0.409955,0.650208,0.126015,0.461726,0.585527,0.200973,0.345227,0.228218,0.798934,...,0.757523,0.588630,0.698973,0.602522,0.068091,0.081960,0.074794,0.579578,0.441237,0.553940


Division into training and test samples

In [4]:
test_size = 0.2
seed = 7
target_feature_name = 'GDP per capita (current US$)'

data_X_all = cleared_data_all.drop([target_feature_name], axis=1)
data_Y_all = cleared_data_all[target_feature_name]
data_X_train_all, data_X_test_all, data_Y_train_all, data_Y_test_all = train_test_split(data_X_all, data_Y_all, test_size=test_size, random_state=seed)

data_X_only_percent = cleared_data_only_percent.drop([target_feature_name], axis=1)
data_Y_only_percent = cleared_data_only_percent[target_feature_name]
data_X_train_only_percent, data_X_test_only_percent, data_Y_train_only_percent, data_Y_test_only_percent = train_test_split(data_X_only_percent, data_Y_only_percent, test_size=test_size, random_state=seed)

datasets = [{'X_train': data_X_train_only_percent, 'X_test': data_X_test_only_percent, 'Y_train': data_Y_train_only_percent,
             'Y_test': data_Y_test_only_percent, 'name': 'only_percent_dataset'},
            {'X_train': data_X_train_all, 'X_test': data_X_test_all,
             'Y_train': data_Y_train_all, 'Y_test':data_Y_test_all, 'name': 'original_data'}]

SVR model

Define grid search params for SVR

In [5]:
SVR_greed_search_params = {
    'C': [i for i in np.linspace(0.1, 100, num=20)],
    'kernel': ['linear', 'poly'],
    'degree': [i for i in range(2, 20)],
    'gamma' : ['scale', 'auto']
}

In [6]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    svr_model = SVR()
    grid_search = GridSearchCV(svr_model, SVR_greed_search_params, n_jobs=7)
    grid_search.fit(dataset.get("X_train"), dataset.get("Y_train"))

    best_params = grid_search.best_params_
    print(f'Best hyper parameters: {best_params}')

    best_svr_model = SVR(
        C=best_params.get('C'),
        kernel=best_params.get('kernel')
    )
    best_svr_model.fit(dataset.get("X_train"), dataset.get("Y_train"))

    test_predict = best_svr_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print(f'RMSE for SVR: ', rmse)
    print(f'R2_score for SVR: ', r2)
    print()

dataset: only_percent_dataset
Best hyper parameters: {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
RMSE for SVR:  0.055518366134471926
R2_score for SVR:  0.8755620912868165

dataset: original_data
Best hyper parameters: {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
RMSE for SVR:  0.059183741409456855
R2_score for SVR:  0.8930817887631275



Random forest model

Define grid search params for Random forest model

In [7]:
random_forest_greed_search_params = {
    'n_estimators': [i for i in range(501, 1002, 250)],
    'criterion' : ["squared_error", "friedman_mse"],
}

In [8]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    random_forest_model = RandomForestRegressor()
    grid_search = GridSearchCV(random_forest_model, random_forest_greed_search_params, n_jobs=7)
    grid_search.fit(dataset.get("X_train"), dataset.get("Y_train"))

    best_params = grid_search.best_params_
    print(f'Best hyper parameters: {best_params}')

    best_random_forest_model = RandomForestRegressor(
        n_estimators=best_params.get('n_estimators'),
        criterion=best_params.get('criterion')
    )
    best_random_forest_model.fit(dataset.get("X_train"), dataset.get("Y_train"))

    test_predict = best_random_forest_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print(f'RMSE for SVR: ', rmse)
    print(f'R2_score for SVR: ', r2)

dataset: only_percent_dataset
Best hyper parameters: {'criterion': 'friedman_mse', 'n_estimators': 751}
RMSE for SVR:  0.03443693784912305
R2_score for SVR:  0.9521228374345768
dataset: original_data
Best hyper parameters: {'criterion': 'squared_error', 'n_estimators': 751}
RMSE for SVR:  0.044095666151647596
R2_score for SVR:  0.9406475477640956


Linear Regression

In [11]:
for dataset in datasets:
    print(f'dataset: {dataset.get("name")}')
    linear_model = LinearRegression()
    linear_model.fit(dataset.get("X_train"), dataset.get("Y_train"))
    test_predict = linear_model.predict(dataset.get("X_test"))
    rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
    r2 = r2_score(dataset.get("Y_test"), test_predict)
    print("RMSE for Linear Regression:", rmse)
    print("R2_score for Linear Regression", r2)
    print()

dataset: only_percent_dataset
RMSE for Linear Regression: 0.04630110379251586
R2_score for Linear Regression 0.9134510029687277

dataset: original_data
RMSE for Linear Regression: 0.07273251723623214
R2_score for Linear Regression 0.8385254478995829



Polynomial Regression (Not working, memory error)

In [8]:
for dataset in datasets:
    r2_scores = []
    rmse_values = []
    print(f'dataset: {dataset.get("name")}')
    for degree in range(2, 20): #  working only with degree=2
        polynomial_features = PolynomialFeatures(degree=degree)
        X_train_poly = polynomial_features.fit_transform(dataset.get("X_train"))
        model = LinearRegression()
        model.fit(X_train_poly, dataset.get("Y_train"))

        test_predict = model.predict(polynomial_features.fit_transform(dataset.get("X_test")))
        rmse_values.append(np.sqrt(mean_squared_error(test_predict, dataset.get("Y_test"))))
        r2_scores.append(r2_score(test_predict, dataset.get("Y_test")))

    max_r2_scores_degree = r2_scores.index(max(r2_scores))
    print(f'Degree with maximum r2: {max_r2_scores_degree+2}')
    print(f"RMSE for Polynomial Regression with degree {max_r2_scores_degree+2}:", r2_scores[max_r2_scores_degree])
    print(f"R2_score for Polynomial Regression with degree {max_r2_scores_degree+2}:", rmse_values[max_r2_scores_degree])
    print()

    max_rmse_degree = rmse_values.index(max(rmse_values))
    print(f'Degree with minimum rmse: {max_rmse_degree+2}')
    print(f"RMSE for Polynomial Regression with degree {max_rmse_degree+2}:", r2_scores[max_rmse_degree])
    print(f"R2_score for Polynomial Regression with degree {max_rmse_degree+2}:", rmse_values[max_rmse_degree])
    print()

dataset: only_percent_dataset
Degree with maximum r2: 2
RMSE for Polynomial Regression with degree 2: 0.9274457730857207
R2_score for Polynomial Regression with degree 2: 0.04213584884115008

Degree with minimum rmse: 2
RMSE for Polynomial Regression with degree 2: 0.9274457730857207
R2_score for Polynomial Regression with degree 2: 0.04213584884115008

dataset: original_data
Degree with maximum r2: 2
RMSE for Polynomial Regression with degree 2: 0.9273541491965996
R2_score for Polynomial Regression with degree 2: 0.048800979714123394

Degree with minimum rmse: 2
RMSE for Polynomial Regression with degree 2: 0.9273541491965996
R2_score for Polynomial Regression with degree 2: 0.048800979714123394



Bagging for regression models

In [20]:
best_models = [
    [
        {'model': SVR(C=0.1, degree=2, gamma='scale', kernel='linear'), 'name': 'svr_model'},
        {'model': LinearRegression(), 'name': 'linear_model'},
    ],
    [
        {'model': SVR(C=0.1, degree=2, gamma='scale', kernel='linear'), 'name': 'svr_model'},
        {'model': LinearRegression(), 'name': 'linear_model'},
    ]
]

for models, dataset in zip(best_models, datasets):
    print(f'dataset: {dataset.get("name")}')
    for model in models:
        bagging_model = BaggingRegressor(model.get('model'), max_samples=200, n_estimators=300)
        bagging_model.fit(dataset.get("X_train"), dataset.get("Y_train"))
        test_predict = bagging_model.predict(dataset.get("X_test"))
        rmse = np.sqrt(mean_squared_error(dataset.get("Y_test"), test_predict))
        r2 = r2_score(dataset.get("Y_test"), test_predict)
        print(f'RMSE for Bagging {model.get("name")}:', rmse)
        print(f'R2_score for Bagging {model.get("name")}:', r2)
        print()

dataset: only_percent_dataset
RMSE for Bagging svr_model: 0.0679845087692796
R2_score for Bagging svr_model: 0.8134053110827588

RMSE for Bagging linear_model: 0.041708297090738554
R2_score for Bagging linear_model: 0.929769741450783

dataset: original_data
RMSE for Bagging svr_model: 0.07143962221573215
R2_score for Bagging svr_model: 0.8442151752742646

RMSE for Bagging linear_model: 0.04459983276339616
R2_score for Bagging linear_model: 0.9392825796265694

